In [20]:
import pandas as pd
from PIL import Image, ImageDraw
import os

In [27]:
path= "../hateful_memes/img/01236.png"
path2= "./modified_images/cropped/01236.png" 
offset=330

In [35]:
arr=[480.3, 72.75+offset, 711.0, 465.1+offset]

In [15]:
def make_object_detection(path):
    with open(path) as f:
        text= f.read()
    maindict=[]
    for val in text.split("\n"):
        if(val==""):
            continue 
        dictv={}
        dictv["file"]=val.split(",")[0].strip()
        dictv["label"]= val.split(",")[1].strip()
        dictv["confidence"]= val.split(",")[2].strip()
        dictv["box"]=val.split("[")[-1][:-1]
        
        maindict.append(dictv)
    return pd.DataFrame(maindict)

In [25]:
cropped_df= make_object_detection("./labels/cropped_labels.txt")
original_df= make_object_detection("./labels/original_labels.txt")
cropped_df

,file,label,confidence,box
0,01235.png,person,1.0,"16.15, 0.22, 543.56, 286.38"
1,01236.png,person,0.944,"480.3, 72.75, 711.0, 465.1"
2,01236.png,person,0.962,"53.13, 114.54, 384.56, 464.73"
3,01236.png,person,0.938,"477.75, 181.41, 710.98, 464.9"
4,01243.png,dog,1.0,"92.23, 74.73, 491.19, 602.85"
...,...,...,...,...
33942,98756.png,person,0.959,"37.89, 0.07, 266.93, 285.07"
33943,98762.png,bowl,0.99,"306.5, 156.33, 531.36, 233.14"
33944,98762.png,dog,0.997,"18.91, 132.26, 134.78, 168.17"
33945,98762.png,person,1.0,"208.88, 0.7, 434.99, 233.87"


In [39]:
dft= cropped_df[cropped_df["file"]=="01236.png"]
dfo= original_df[original_df["file"]=="01236.png"]

In [87]:
def find_area(arr):
    if(arr[0]>arr[2] or arr[1]>arr[3]):
        return 0
    else:
        return (arr[2]-arr[0])*(arr[3]-arr[1])

def find_iou(str1, str2, offset):
    arr1= [float(val) for val in str1.split(",")]
    if(offset!=None):
        arr1[1]+=offset
        arr1[3]+=offset
    arr2= [float(val) for val in str2.split(",")]
    union=[min(arr1[0], arr2[0]), min(arr1[1], arr2[1]), max(arr1[2], arr2[2]), max(arr1[3], arr2[3])]
    intersection=[max(arr1[0], arr2[0]), max(arr1[1], arr2[1]), min(arr1[2], arr2[2]), min(arr1[3], arr2[3])]
    
    if(find_area(union)!=0):
        iou_val= (find_area(intersection)/find_area(union))
    else:
        iou_val=0
    return iou_val
def myfunc(e):
    return e[1]

In [50]:
dict_info={}
for j in range(len(dft)):
    maxval=0
    maxarea=0
    print(j)
    for i in range(len(dfo)):
        print(i)
        new_area= find_iou(dft.iloc[j].box, dfo.iloc[i].box)
        print(new_area)
    print("\n")

0
0
0.0
1
0.08357823325573562
2
0.0
3
0.0
4
0.0
5
0.0
6
0.011231303355698795
7
0.0
8
0.014496750246880132
9
0.32817180394107415
10
0.0


1
0
0.0
1
0.0
2
0.15402868961405072
3
0.13523898870144935
4
0.11948549308639482
5
0.0203673416480884
6
0.027636607613501674
7
0.0
8
0.006533613128570455
9
0.0
10
0.0


2
0
0.0
1
0.09917053751409988
2
0.0
3
0.0
4
0.0
5
0.0
6
0.013573863637564534
7
0.0
8
0.009314295074671955
9
0.16797932195642368
10
0.0




In [52]:
dict_info={}
for j in range(len(dft)):
    maxval=0
    maxarea=0
    print(j)
    for i in range(len(dfo)):
        print(i)
        new_area= find_iou(dft.iloc[j].box, dfo.iloc[i].box, offset)
        print(new_area)
    print("\n")

0
0
0.0
1
0.9774746199104996
2
0.0
3
0.0
4
0.0
5
0.0
6
0.155494453564253
7
0.7601570153058874
8
0.0
9
0.0
10
0.0


1
0
0.0
1
0.0
2
0.0
3
0.0
4
0.0
5
0.6694361222488138
6
0.3453315363239001
7
0.0
8
0.0
9
0.0
10
0.0


2
0
0.0
1
0.7180111977464654
2
0.0
3
0.0
4
0.0
5
0.0
6
0.12322946228719765
7
0.9377096713537304
8
0.0
9
0.0
10
0.0




In [46]:
dict_info

{}

In [53]:
dft

,file,label,confidence,box
1,01236.png,person,0.944,"480.3, 72.75, 711.0, 465.1"
2,01236.png,person,0.962,"53.13, 114.54, 384.56, 464.73"
3,01236.png,person,0.938,"477.75, 181.41, 710.98, 464.9"


In [93]:
arr2=[52.95, 445.42, 285.75, 779.61]

In [94]:
arr=[53.13, 114.54+offset, 384.56, 464.73+offset]

In [95]:
a2= (", ").join([str(fl) for fl in arr])
a1= (", ").join([str(fl) for fl in arr2])

In [96]:
find_iou(a2, a1, offset=None)

0.6694361222488138

In [120]:
tempdf= []
with open("../outputs/crop_offset.txt") as f:
    text= f.read()
for val in text.split("\n"):
    if(val==""):
        continue
    dictv={}
    dictv["file"]=os.path.basename(val.split(", ")[0])
    dictv["offset"]=float(val.split(", ")[1])
    tempdf.append(dictv)
tempdf=pd.DataFrame(tempdf)
tdf= cropped_df.merge(tempdf)
tdf[tdf["file"]=="98762.png"]

,file,label,confidence,box,offset
33943,98762.png,bowl,0.99,"306.5, 156.33, 531.36, 233.14",68.0
33944,98762.png,dog,0.997,"18.91, 132.26, 134.78, 168.17",68.0
33945,98762.png,person,1.0,"208.88, 0.7, 434.99, 233.87",68.0
